In [1]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

plt.style.use('dark_background')

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [8]:

train_data = datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = ToTensor(), 
    download = True,            
)
test_data = datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = ToTensor()
)

In [4]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [11]:
from torch.utils.data import DataLoader
loaders = {
    'train' : torch.utils.data.DataLoader(train_data, 
                                          batch_size=100, 
                                          shuffle=True, 
                                          num_workers=1),
    
    'test'  : torch.utils.data.DataLoader(test_data, 
                                          batch_size=100, 
                                          shuffle=True, 
                                          num_workers=1),
}
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x289d43e6290>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x289d44062d0>}

In [3]:
import torch.nn as nn
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.Tanh(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.Tanh(),                      
            nn.MaxPool2d(2),                
        )        
        # fully connected layer, output 10 classes
        self.out = nn.Linear(32 * 7 * 7, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)        
        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output, x    # return x for visualization

In [4]:
cnn = CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Tanh()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Tanh()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


In [8]:
loss_func = nn.CrossEntropyLoss()   
loss_func

CrossEntropyLoss()

In [9]:
from torch import optim
optimizer = optim.Adam(cnn.parameters(), lr = 0.01)   
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    weight_decay: 0
)

In [ ]:
from torch.autograd import Variable
num_epochs = 10

cnn.train()

# Train the model
total_step = len(loaders['train'])

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(loaders['train']):

        # gives batch data, normalize x when iterate train_loader
        b_x = Variable(images)   # batch x
        b_y = Variable(labels)   # batch y
        output = cnn(b_x)[0]               
        loss = loss_func(output, b_y)

        # clear gradients for this training step   
        optimizer.zero_grad()           

        # backpropagation, compute gradients 
        loss.backward()                
        # apply gradients             
        optimizer.step()                

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/10], Step [100/600], Loss: 0.2917
Epoch [1/10], Step [200/600], Loss: 0.1751
Epoch [1/10], Step [300/600], Loss: 0.1775
Epoch [1/10], Step [400/600], Loss: 0.1899
Epoch [1/10], Step [500/600], Loss: 0.0800
Epoch [1/10], Step [600/600], Loss: 0.0699
Epoch [2/10], Step [100/600], Loss: 0.0294
Epoch [2/10], Step [200/600], Loss: 0.1879
Epoch [2/10], Step [300/600], Loss: 0.2281
Epoch [2/10], Step [400/600], Loss: 0.1701
Epoch [2/10], Step [500/600], Loss: 0.0185
Epoch [2/10], Step [600/600], Loss: 0.5865


In [12]:
cnn.eval()    
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in loaders['test']:
        break
        test_output, last_layer = cnn(images)
        pred_y = torch.max(test_output, 1)[1].data.squeeze()
        accuracy = (pred_y == labels).sum().item() / float(labels.size(0))

In [ ]:
torch.save(cnn.state_dict(), 'model/cnn.torch')

In [6]:
cnn = CNN()
cnn.load_state_dict(torch.load('model/cnn.torch'))
cnn.eval()

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Tanh()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Tanh()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)

In [37]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import pickle

In [44]:
x = np.array(train_data.data).reshape(-1, 784)
y = np.array(train_data.targets)

In [49]:
rf = RandomForestClassifier()
rf.fit(x, y)

RandomForestClassifier()

In [50]:
with open('rf.pickle', 'wb') as fw:
    pickle.dump(rf, fw)

In [51]:
with open('rf.pickle', 'rb') as fr:
    rf = pickle.load(fr)

In [52]:
xtest = np.array(test_data.data).reshape(-1, 784)
ytest = np.array(test_data.targets)

In [53]:
pre = rf.predict(xtest)

In [56]:
sum(pre == ytest) / len(ytest)

0.9691